# Assignment 9 (Exercise 9.3)

## Simon Laurent Lebailly, 2549365, s9sileba@teams.uni-saarland.de
## Christian Mathieu Schmidt, 2537621, s9cmscmi@teams.uni-saarland.de



### In the lecture you have been introduced to VGG16. For this problem your task is to implement a VGG like CNN architecture for classification on the CIFAR10 dataset.

In [62]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

### 1. Load the dataset (0.5 point)
To load the dataset, you can use the inbuilt dataloader for CIFAR10 provided in the torchvision package. Load both test set and trainset separately. Define the transformations you might need to load the data appropriately.

In [63]:
#Import datasets from torchvision to import the CIFAR10 dataset
from torchvision import datasets


#Check if CUDA is available, if not use the CPU. 
#We have worked on a PC (CUDA available) and a Macbook (CUDA not available), so we have used both.
#Therefore we have implemented a case distinction, and we have modified the code where necessary.
train_on_GPU = torch.cuda.is_available()
device = torch.device('cuda' if train_on_GPU else 'cpu')

if train_on_GPU:
    print('CUDA available!')
else:
    print('CUDA not available!')


#Hyperparameters:
batch_size = 4
learning_rate = 0.01



transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

trainset = datasets.CIFAR10('data', train=True, download=True, transform=transform)
quantity_train = len(trainset)
print('Quantity training data: '+ str(quantity_train))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)


testset = datasets.CIFAR10('data', train=False, download=True, transform=transform)
quantity_test = len(testset)
print('Quantity test data: '+ str(quantity_test))

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


CUDA not available!
Files already downloaded and verified
Quantity training data: 50000
Files already downloaded and verified
Quantity test data: 10000


### Create the model architecture (1.0 point)
Implement the class below such that the final architecture follows the same pattern of layers as VGG16.

In [64]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #Pooling filter with size 2x2, to halve the size of the image
        self.pool = nn.MaxPool2d(2,2)        
        
        #First VGG layer with two times Convolution and duplicating of the features
        self.conv11 = nn.Conv2d(3,32,3)
        self.conv12 = nn.Conv2d(32,64,14,stride=2)
        
        #Second VGG layer with three times Convolution and duplicating of the features
        self.conv21 = nn.Conv2d(16,32,3)
        self.conv22 = nn.Conv2d(32,64,5,stride=2)
        self.conv23 = nn.Conv2d(64,32,3)
        
        #Third VGG layer with fully connected layers
        self.fc1 = nn.Linear(32*4*4, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 10)
        

    def forward(self, x):
        #First VGG layer with two times Convolution + ReLu and duplicating of the features
        #Pooling filter with size 2x2, to halve the size of the image, after the second convolution
        out = F.relu(self.conv11(x))
        print(out.size())
        out = F.relu(self.conv12(out))
        print(out.size())
        
        #Second VGG layer with three times Convolution + ReLu and duplicating of the features
        #Pooling filter with size 2x2, to halve the size of the image, after the third convolution
        out = F.relu(self.conv21(out))
        out = F.relu(self.conv22(out))
        out = self.pool(F.relu(self.conv23(out)))
        
        #Third VGG layer with fully connected layers
        out = out.view(-1, 32*4*4)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        
        #We don't use SOFTMAX in the forward process, because we use CrossEntropyLoss() as criterion
        
        return out


net = Net().to(device)


### Loss function and optimizer (0.5 point)
Define the loss function and optimizer to train the model.

In [65]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

### Train the model (1.0 point)

In [66]:
#Import matplotlib to show the course of the loss
import matplotlib.pyplot as plt

#Two lists  only used for the plot and the statistics
lossdata = []
mean_lossdata = []

#We have used 16 epochs, because we had the best results (>= 70% accuracy) in dependence of the time the CNN needs to train (ca. 20min).
epochs = 1

for epoch in range(epochs):
    for i, data in enumerate(trainloader, 0):
        #"Transport" the data to CUDA if available, and otherwise to the CPU
        image = data[0].to(device)
        label = data[1].to(device)
        
        #Compute the forward propagation
        output = net(image)
        
        #Compute the loss
        loss = criterion(output, label)
        
        #Reset gradient and execute backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #Print statistics
        if ((i+1) % 10000 == 0) or ((i+1) % (50000/batch_size) == 0):
            lossdata.append(loss.item())
            mean_loss = sum(lossdata)/len(lossdata)
            mean_lossdata.append(mean_loss)
            
            print("Progress: " + str(round(((((i+1)+epoch*12000)*100)/((50000/batch_size)*epochs)), 2)) + "%, Epoch: " + str(epoch+1) 
                  + ", Datasets: " + str((i+1)+epoch*12000) + ", Loss: " + str(loss.item()) + ", Mean Loss: " + str(mean_loss))


print('Finished Training')

#Plot the loss function
plt.title("Loss during the training of the CNN")
plt.plot(lossdata,'-b',label='Loss')
plt.plot(mean_lossdata,'-r',label='Mean Loss')
plt.xlabel('Number of datasets divided by 1000')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()


torch.Size([4, 32, 30, 30])
torch.Size([4, 64, 9, 9])


RuntimeError: Given groups=1, weight of size [32, 16, 3, 3], expected input[4, 64, 9, 9] to have 16 channels, but got 64 channels instead

Code below generates the class wise accuracy of the model. You can use the results from the code below to decide the values of hyperparametrs.

In [ ]:

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        
        #We have set .cuda() to .to(device) because of the different systems we have used.
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


total_class = 0
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
    total_class += class_correct[i]


#We have added the value of the overall accuracy.
total = 100 * total_class / (len(testloader)*batch_size)

print(" ")
print('Accuracy total: ' + str(total) + '%')
